In [1]:
import tarfile
import os
import pandas as pd
from pathlib import Path
from datetime import datetime


In [2]:

# 设置数据获取参数
params = {
    # 数据获取配置
    'begin_date': '2024-06-01',
    'end_date': '2025-04-03',
    'time_intervals': ['1m'],  # 可选: ['1m', '5m', '15m', '1h', '4h', '1d']
    'selected_symbols': [
            "KASUSDT",
            "INJUSDT",
            "NEARUSDT",
            "SOLUSDT",
            "1000PEPEUSDT",
            "AAVEUSDT",
            "APTUSDT",
            "SUIUSDT",
        ],
    # 'selected_symbols': [
    #         "BTCUSDT",
    #         "ETHUSDT",
    #         "SOLUSDT",
    #         "XRPUSDT",
    #         "DOGEUSDT",
    #         "BNBUSDT",
    #         "NEARUSDT",
    #         "ADAUSDT",
    #         "EOSUSDT",
    #         "LTCUSDT",
    #         "SUIUSDT",
    #         "1000PEPEUSDT",
    #         "AVAXUSDT",
    #         "LINKUSDT",
    #         "AAVEUSDT",
    #         "TRXUSDT",
    #         "ICPUSDT",
    #         "WLDUSDT",
    #         "DOTUSDT",
    #         "APTUSDT",
    #         "UNIUSDT",
    #         "FILUSDT",
    #         "RAYUSDT",
    #         "1000SHIBUSDT",
    #         "SEIUSDT",
    #         "ARBUSDT",
    #         "ATOMUSDT",
    #         "LDOUSDT",
    #         "INJUSDT",
    #         "OPUSDT"
    #     ],
    # ... 现有代码 ...
    'base_path': r'\\znas\Main\futures',  # 指定数据保存的根目录
    'tar_filename': 'kasusdt.tar'  # 指定tar文件名（改为 .tar 而不是 .tar.gz）
}


In [3]:
def save_selected_to_tar(params):
    """
    将指定代币的CSV文件打包成一个tar文件，支持增量更新
    同时删除不在selected_symbols中的代币文件和日期范围外的文件
    """
    from tqdm import tqdm

    csv_directory = params['base_path']
    tar_filename = os.path.join(csv_directory, params['tar_filename'])
    
    print(f"\n开始处理数据打包任务...")
    print(f"数据源目录: {csv_directory}")
    print(f"目标文件: {tar_filename}")
    
    # 创建交易对名称映射
    symbol_mapping = {}
    for symbol in params['selected_symbols']:  # 修正这里的键名
        # 不再需要复杂转换，直接加上"_USDT"后缀
        symbol_mapping[symbol] = f"{symbol}_USDT"
    
    # 获取日期范围
    start_date = datetime.strptime(params['begin_date'], '%Y-%m-%d')
    end_date = datetime.strptime(params['end_date'], '%Y-%m-%d')
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    date_strs = [date.strftime('%Y-%m-%d') for date in date_range]
    
    print(f"\n时间范围: {params['begin_date']} 至 {params['end_date']}")
    print(f"需要处理 {len(date_range)} 天的数据")
    print(f"交易对数量: {len(params['selected_symbols'])}")
    print(f"时间间隔: {params['time_intervals']}")
    
    # 创建需要保留的文件列表
    keep_files = set()
    for symbol in params['selected_symbols']:
        symbol_name = symbol_mapping[symbol]
        for date_str in date_strs:
            for interval in params['time_intervals']:
                file_name = f"{date_str}_{symbol_name}_{interval}.csv"
                file_path = os.path.join(date_str, file_name)
                # 将Windows路径分隔符转换为POSIX格式(/)
                file_path = file_path.replace('\\', '/')
                keep_files.add(file_path)
    
    # 获取现有tar文件中的文件列表，并确定需要删除的文件
    existing_files = set()
    files_to_remove = set()
    
    if os.path.exists(tar_filename):
        print("\n发现现有tar文件，将进行清理和更新...")
        with tarfile.open(tar_filename, "r") as tar:
            existing_files = {member.name for member in tar.getmembers()}
            # 找出需要删除的文件
            files_to_remove = existing_files - keep_files
            # 打印统计信息
            print(f"现有tar文件包含 {len(existing_files)} 个文件")
            print(f"需要删除的文件数: {len(files_to_remove)}")
            print(f"需要保留的文件数: {len(existing_files - files_to_remove)}")
                
        # 判断tar文件是否需要重建
        if not existing_files:
            print("警告：tar文件中没有找到任何文件，可能是空文件或文件损坏")
            print("将创建新文件")
            mode = "w"  # 强制重写模式
        else:
            if files_to_remove:
                print("将创建新的tar文件并只包含需要保留的文件...")
                mode = "w"  # 重写模式
            else:
                mode = "a"  # 如果没有需要删除的文件，使用追加模式
    else:
        print("\n未发现现有tar文件，将创建新文件...")
        mode = "w"  # 新建模式
    
    # 收集需要添加的文件
    print("\n开始扫描需要处理的文件...")
    files_to_add = []
    total_expected = len(params['selected_symbols']) * len(date_range) * len(params['time_intervals'])
    missing_count = 0
    
    progress_bar = tqdm(total=total_expected, desc="扫描文件")
    
    # 遍历每个交易对
    for symbol in params['selected_symbols']:
        symbol_name = symbol_mapping[symbol]
        for date in date_range:
            date_str = date.strftime('%Y-%m-%d')
            for interval in params['time_intervals']:
                progress_bar.update(1)
                
                # 构建文件名和路径
                file_name = f"{date_str}_{symbol_name}_{interval}.csv"
                file_path = os.path.join(csv_directory, date_str, file_name)
                
                # 检查文件状态
                if os.path.exists(file_path):
                    relative_path = os.path.join(date_str, file_name)
                    # 转换为POSIX格式路径
                    relative_path = relative_path.replace('\\', '/')
                    
                    # 如果是新建模式或文件不在tar中，添加到需要添加的列表
                    if mode == "w" or relative_path not in existing_files:
                        files_to_add.append((file_path, relative_path))
                else:
                    missing_count += 1
    progress_bar.close()
    
    # 打印扫描结果
    print("\n文件扫描完成:")
    if mode == "w":
        print(f"- 将重建tar文件，添加 {len(files_to_add)} 个文件")
    else:
        print(f"- 需要新增的文件: {len(files_to_add)}")
    print(f"- 缺失的文件: {missing_count}")
    
    # 创建/更新tar文件
    if mode == "w" or files_to_add:
        print(f"\n开始{'创建' if mode == 'w' else '更新'}tar文件...")
        with tarfile.open(tar_filename, mode) as tar:
            for file_path, arcname in tqdm(files_to_add, desc="打包文件"):
                tar.add(file_path, arcname=arcname)
        
        total_files = len(files_to_add) if mode == "w" else (len(existing_files) - len(files_to_remove) + len(files_to_add))
        print(f"\n更新完成:")
        print(f"- 当前总文件数: {total_files}")
        print(f"- 理论总文件数: {total_expected}")
        print(f"- 数据完整率: {(total_files/total_expected)*100:.2f}%")
        print(f"- 包含 {len(params['selected_symbols'])} 个交易对的数据")
        print(f"\n文件已保存到:")
        print(f"{tar_filename}")
        
        if os.path.exists(tar_filename):
            size_mb = os.path.getsize(tar_filename) / (1024 * 1024)
            print(f"文件大小: {size_mb:.2f} MB")
    else:
        print("\n没有需要更新的文件")


In [4]:
# 运行打包程序
if __name__ == "__main__":
    save_selected_to_tar(params)


开始处理数据打包任务...
数据源目录: \\znas\Main\futures
目标文件: \\znas\Main\futures\suiusdt.tar

时间范围: 2024-06-01 至 2025-04-03
需要处理 307 天的数据
交易对数量: 1
时间间隔: ['1m']

未发现现有tar文件，将创建新文件...

开始扫描需要处理的文件...


扫描文件: 100%|██████████| 307/307 [00:00<00:00, 549.76it/s]



文件扫描完成:
- 将重建tar文件，添加 307 个文件
- 缺失的文件: 0

开始创建tar文件...


打包文件: 100%|██████████| 307/307 [00:13<00:00, 22.09it/s]


更新完成:
- 当前总文件数: 307
- 理论总文件数: 307
- 数据完整率: 100.00%
- 包含 1 个交易对的数据

文件已保存到:
\\znas\Main\futures\suiusdt.tar
文件大小: 24.60 MB
